In [6]:
import pandas as pd
import numpy as np
from collections import Counter

In [7]:
df = pd.read_csv('bike data.csv', encoding='unicode_escape').astype(str)
df_copy = df.copy()

In [8]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335075 entries, 0 to 335074
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ride_id             335075 non-null  object
 1   rideable_type       335075 non-null  object
 2   started_at          335075 non-null  object
 3   started_day         335075 non-null  object
 4   started_time        335075 non-null  object
 5   ended_at            335075 non-null  object
 6   ended_day           335075 non-null  object
 7   ended_time          335075 non-null  object
 8   start_station_name  335075 non-null  object
 9   start_station_id    335075 non-null  object
 10  end_station_name    335075 non-null  object
 11  end_station_id      335075 non-null  object
 12  start_lat           335075 non-null  object
 13  start_lng           335075 non-null  object
 14  end_lat             335075 non-null  object
 15  end_lng             335075 non-null  object
 16  me

In [9]:
datetime_cols = ['started_at', 'ended_at']
for col in datetime_cols:
  df_copy[col] = pd.to_datetime(df_copy[col])

In [10]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335075 entries, 0 to 335074
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             335075 non-null  object        
 1   rideable_type       335075 non-null  object        
 2   started_at          335075 non-null  datetime64[ns]
 3   started_day         335075 non-null  object        
 4   started_time        335075 non-null  object        
 5   ended_at            335075 non-null  datetime64[ns]
 6   ended_day           335075 non-null  object        
 7   ended_time          335075 non-null  object        
 8   start_station_name  335075 non-null  object        
 9   start_station_id    335075 non-null  object        
 10  end_station_name    335075 non-null  object        
 11  end_station_id      335075 non-null  object        
 12  start_lat           335075 non-null  object        
 13  start_lng           335075 no

In [11]:
# Cria colunas só para data e horário. Além disso, faz uma subtração que gera o tempo total

df_copy['started_day'] = df_copy['started_at'].dt.date
df_copy['started_hour'] = df_copy['started_at'].dt.time

df_copy['end_day'] = df_copy['ended_at'].dt.date
df_copy['end_hour'] = df_copy['ended_at'].dt.time

df_copy['total_time'] = df_copy['ended_at'] - df_copy['started_at']

In [12]:
mean_time = df_copy['total_time'].mean()
print(f"Mean time: {mean_time}")

Mean time: 0 days 00:13:26.343398725


In [13]:
max_time = df_copy['total_time'].max()
print(f"Max time: {max_time}")

Max time: 1 days 01:00:51.765000


In [14]:
# Tempo mínimo está errado

min_time = df_copy['total_time'].min()
print(f"Min time: {min_time}")

Min time: -1 days +23:03:58.698000


In [15]:
# Observando as informações de min, nota-se que o started_at e o ended_at podem estar trocados

min_time_row = df_copy[df_copy['total_time'] == df_copy['total_time'].min()]
print(min_time_row)

                ride_id  rideable_type              started_at started_day  \
86015  1ED3B2ACE0FE3C99  electric_bike 2024-11-03 01:59:24.648  2024-11-03   

       started_time                ended_at   ended_day    ended_time  \
86015  01:59:24.648 2024-11-03 01:03:23.346  2024-11-03  01:03:23.346   

      start_station_name start_station_id  ... start_lat start_lng   end_lat  \
86015                nan              nan  ...     41.92    -87.67  41917805   

         end_lng member_casual weekday     started_hour     end_day  \
86015  -87653449        member       1  01:59:24.648000  2024-11-03   

              end_hour               total_time  
86015  01:03:23.346000 -1 days +23:03:58.698000  

[1 rows x 22 columns]


In [16]:
# Corrige o problema

mask = df_copy['started_at'] > df_copy['ended_at']

df_copy.loc[mask, ['started_at', 'ended_at']] = df_copy.loc[mask, ['ended_at', 'started_at']].values

df_copy['total_time'] = df_copy['ended_at'] - df_copy['started_at']

In [17]:
test = df_copy[df_copy['ride_id']=='1ED3B2ACE0FE3C99']
print(test)
new_min_time = df_copy['total_time'].min()
print(f'Min time: {new_min_time}')

                ride_id  rideable_type              started_at started_day  \
86015  1ED3B2ACE0FE3C99  electric_bike 2024-11-03 01:03:23.346  2024-11-03   

       started_time                ended_at   ended_day    ended_time  \
86015  01:59:24.648 2024-11-03 01:59:24.648  2024-11-03  01:03:23.346   

      start_station_name start_station_id  ... start_lat start_lng   end_lat  \
86015                nan              nan  ...     41.92    -87.67  41917805   

         end_lng member_casual weekday     started_hour     end_day  \
86015  -87653449        member       1  01:59:24.648000  2024-11-03   

              end_hour             total_time  
86015  01:03:23.346000 0 days 00:56:01.302000  

[1 rows x 22 columns]
Min time: 0 days 00:00:00.192000


In [18]:
# Novo tempo mínimo é muito baixo e provavelmente acidental ou fruto de erro

min_time_row = df_copy[df_copy['total_time'] == df_copy['total_time'].min()]
print(min_time_row)

                 ride_id  rideable_type              started_at started_day  \
288666  CB3CD92ADE4BA750  electric_bike 2024-11-23 06:51:26.965  2024-11-23   

        started_time                ended_at   ended_day    ended_time  \
288666  06:51:26.965 2024-11-23 06:51:27.157  2024-11-23  06:51:27.157   

       start_station_name start_station_id  ... start_lat start_lng end_lat  \
288666                nan              nan  ...     41.95    -87.68   41.95   

       end_lng member_casual weekday     started_hour     end_day  \
288666  -87.68        member       7  06:51:26.965000  2024-11-23   

               end_hour             total_time  
288666  06:51:27.157000 0 days 00:00:00.192000  

[1 rows x 22 columns]


In [19]:
# Filtra o tempo mínimo para que o usuário tenha mais de 45 segundos ativos

df_copy = df_copy[df_copy['total_time'] >= pd.Timedelta(minutes=0.75)]
print(df_copy['total_time'].min())

0 days 00:00:45.018000
